# 🎤 GPT-SoVITS + vTTS API 테스트 (Kaggle)

**목표**: GPT-SoVITS 음성 클로닝 TTS를 vTTS API로 테스트

---

## 📌 사전 요구사항
- Kaggle GPU 활성화 필수! (Settings → Accelerator → GPU T4 x2)
- 예상 시간: 설치 ~10-15분, 테스트 ~5분

## 1️⃣ 환경 설정 및 설치 (~10-15분)

In [ ]:
# GPT-SoVITS 설치 (시간이 오래 걸립니다 ~10-15분)
import os
import subprocess
import sys

print("🔧 GPT-SoVITS 환경 설정 중...")
print("   ⚠️ 이 과정은 10-15분 정도 소요됩니다.\n")

# 1. numpy + onnxruntime 완전 재설치 (순서 중요!)
print("   [1/6] numpy + ONNX Runtime 재설치 (호환성 보장)...")
!pip uninstall numpy onnxruntime onnxruntime-gpu -y -q
!pip install -q "numpy>=1.24.0,<2.0.0" onnxruntime-gpu

# 2. vTTS + Supertonic 설치 (GPU 자동 지원, numpy는 재설치 안함)
print("   [2/6] vTTS + Supertonic 설치 (GPU 자동 지원)...")
!pip install -q "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git" --no-deps
!pip install -q fastapi uvicorn httpx "pydantic>=2.0" pydantic-settings loguru soundfile librosa huggingface-hub

# 3. CUDA 지원 확인
print("   [3/6] CUDA 지원 확인...")
import onnxruntime as ort
providers = ort.get_available_providers()
print(f"      ONNX Providers: {providers}")
if "CUDAExecutionProvider" in providers:
    print("      🎉 CUDA 지원 활성화!")
else:
    print("      ⚠️ CUDA 미지원 (CPU 모드)")

# 4. GPT-SoVITS 저장소 클론
print("   [4/6] GPT-SoVITS 저장소 클론...")
GPT_SOVITS_PATH = os.path.expanduser("~/.vtts/GPT-SoVITS")
os.makedirs(os.path.dirname(GPT_SOVITS_PATH), exist_ok=True)

if not os.path.exists(GPT_SOVITS_PATH):
    !git clone --depth 1 https://github.com/RVC-Boss/GPT-SoVITS.git {GPT_SOVITS_PATH}
else:
    print(f"      GPT-SoVITS already exists: {GPT_SOVITS_PATH}")

# 5. GPT-SoVITS requirements 설치
print("   [5/6] GPT-SoVITS 의존성 설치 (오래 걸림)...")
!pip install -q -r {GPT_SOVITS_PATH}/requirements.txt

# 6. 환경변수 설정
print("   [6/6] 환경변수 설정...")
os.environ["GPT_SOVITS_PATH"] = GPT_SOVITS_PATH

print(f"\n✅ GPT-SoVITS 설치 완료!")
print(f"   경로: {GPT_SOVITS_PATH}")

## 2️⃣ 참조 오디오 생성 (Supertonic 사용)

In [ ]:
# 참조 오디오 생성 (GPT-SoVITS는 참조 오디오가 필수!)
from vtts.engines.supertonic import SupertonicEngine
from vtts.engines.base import TTSRequest
import IPython.display as ipd
from pathlib import Path
import soundfile as sf

print("🎙️ 참조 오디오 생성 중 (Supertonic 사용)...\n")

# Supertonic 엔진 로드
engine = SupertonicEngine(model_id="Supertone/supertonic-2", device="cuda")
engine.load_model()

# 참조 오디오 저장 디렉토리
ref_audio_dir = Path("./reference_audio")
ref_audio_dir.mkdir(exist_ok=True)

# 한국어 참조 오디오 생성
try:
    ko_request = TTSRequest(
        text="안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다.",
        language="ko",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    ko_output = engine.synthesize(ko_request)
    ko_path = ref_audio_dir / "reference_ko.wav"
    sf.write(ko_path, ko_output.audio, ko_output.sample_rate)
    print(f"✅ 한국어 참조 오디오: {ko_path}")
    print(f"   내용: {ko_request.text}")
except Exception as e:
    print(f"⚠️ 참조 오디오 생성 실패: {e}")
    print("   직접 오디오 파일을 업로드하세요!")
    ko_path = None

# 영어 참조 오디오 생성
try:
    en_request = TTSRequest(
        text="Hello, this is a reference audio for voice cloning.",
        language="en",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    en_output = engine.synthesize(en_request)
    en_path = ref_audio_dir / "reference_en.wav"
    sf.write(en_path, en_output.audio, en_output.sample_rate)
    
    # 엔진 언로드 (메모리 절약)
    engine.unload_model()
    
    print(f"\n✅ 영어 참조 오디오: {en_path}")
    print(f"   내용: {en_request.text}")
except Exception as e:
    print(f"⚠️ 영어 참조 오디오 생성 실패: {e}")
    en_path = None

print(f"\n📁 참조 오디오가 저장되었습니다: {ref_audio_dir.absolute()}")